This script will be for linking the siteIDs to the catchment IDs - which are either NHDPlusIDs,
gridcodes, or new catIDs made from the gridcodes for BB. This will give us a way to link the
thermal sensitivity responses by year to the covariates. This script also includes code to
check the watersheds. The watersheds are individual feature classes to avoid overlapping
polygons so the best way to check they are correct is to 1) check that every siteID/catID
has a watershed, and 2) check that the watershed area approximately matches the watershed
area from the flow accumulation grid. They won't be exact for sites that are not close to
the catchment outlet and could be quite different for very small watersheds in some cases.

# Watershed Summaries

1. read in all watersheds feature classes
2. create a table with the NHDPlusID/catID of the watershed name, region, and watershed area
3. merge original point feature classes (inside and outside bb)
4. do a spatial join with merged catchments by region to get catID and region on the points dataset
5. clip to the region and save in each regional gdb
6. merge all point feature classes with siteIDs and catIDs into one table
7. join the watershed area from the watershed fcs to the points table using the catID
NOT DONE and not sure we need it: 8. join the watershed area from the fac grid to the points table using the catID
9. compare the results to ensure that watersheds have been created correctly

In [21]:
# steps 1 and 2

import arcpy
import os

import numpy
import pandas as pd

regions = ["Kodiak", "Copper_River", "Prince_William_Sound", "Cook_Inlet", "Bristol_Bay"]
wtdList = []

for region in regions:
    local_gdb = "W:\\GIS\\AKSSF\\" + region + "\\" + region + ".gdb\\Watersheds"
    arcpy.env.workspace = local_gdb
    wtds = arcpy.ListFeatureClasses()
    print(region + ": " + str(len(wtds)) + " watersheds")

    for wtd in wtds:
        wtdName = wtd[4:20]
        print("Starting wtd: " + wtdName)
        wtdPath = os.path.join(arcpy.env.workspace, wtd)
        # field_names = [f.name for f in arcpy.ListFields(wtdPath)]
        # print(field_names)
        # if "Area_km2" in field_names:
        #     print("Area already calculated")
        # else:
        #     arcpy.AddField_management(wtdPath, "Area_km2", "DOUBLE")
        #     expression1 = "{0}".format("!SHAPE.area@SQUAREKILOMETERS!")
        #     arcpy.CalculateField_management(wtdPath, "Area_km2", expression1, "PYTHON", )
        wtdArea = [row[0] for row in arcpy.da.SearchCursor(wtdPath, ['Area_km2'])]
        # print("wtdName: " + str(wtdArea))
        wtdList.append({'Region': region, 'cat_ID': wtdName, 'Area_km2': wtdArea})

wtdDf = pd.DataFrame(wtdList)
print(wtdDf)


Kodiak: 28 watersheds


TypeError: can only concatenate str (not "list") to str

In [4]:
#merge watersheds into one feature class for each region

import arcpy
import os
import pandas as pd
arcpy.env.overwriteOutput = True

regions = ["Kodiak", "Copper_River", "Prince_William_Sound", "Cook_Inlet", "Bristol_Bay"]

for region in regions:
    local_gdb = "W:\\GIS\\AKSSF\\" + region + "\\" + region + ".gdb"
    arcpy.env.workspace = local_gdb + "\\Watersheds"
    wtds = arcpy.ListFeatureClasses()
    print(region + ": " + str(len(wtds)) + " watersheds")

    #add cat_ID to each watershed before merging
    for wtd in wtds:
        wtdName = wtd[4:20]
        wtdPath = os.path.join(arcpy.env.workspace, wtd)
        arcpy.AddField_management(wtdPath, "cat_ID", "DOUBLE")
        # expression1 = "{0}".format("!SHAPE.area@SQUAREKILOMETERS!")
        arcpy.CalculateField_management(wtdPath, "cat_ID", wtdName, "PYTHON")

    arcpy.env.workspace = local_gdb
    wtdMerge = [local_gdb + "\\Watersheds\\" + s for s in wtds]
    wtd_output = "wtds_merge"
    arcpy.Merge_management(wtdMerge, wtd_output)


Kodiak: 28 watersheds
Copper_River: 28 watersheds
Prince_William_Sound: 19 watersheds
Cook_Inlet: 241 watersheds
Bristol_Bay: 114 watersheds


In [ ]:
print(len(wtdDf))
print(len(wtdList))


# step 3 - done

import arcpy

gdb = "W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb"
arcpy.env.workspace = gdb
bb_pts = gdb + "\\bb_md_verified_DM"
other_pts = gdb + "\\sites_outside_bb_verified_DM"

output = "akssf_pts_verified"
arcpy.Merge_management([bb_pts, other_pts], output)

In [ ]:
# steps 4-6

import arcpy
import os
arcpy.env.overwriteOutput = True
regions_dict = {"Kodiak": '!gridcode!', "Copper_River": '!NHDPlusID!', "Prince_William_Sound": '!gridcode!', "Cook_Inlet": '!NHDPlusID!', "Bristol_Bay": '!catID!'}
# regions_dict = {"Bristol_Bay": '!catID!'}

points = "W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb\\akssf_pts_verified"
# sites_lst = []
cats_lst = []

for key, value in regions_dict.items():
    arcpy.env.workspace = "W:\\GIS\\AKSSF\\" + key + "\\" + key + ".gdb"
    cats = os.path.join(arcpy.env.workspace, "cats_merge")
    # arcpy.Clip_analysis(points, cats, r"memory\sites_clip")
    # arcpy.SpatialJoin_analysis(r"memory\sites_clip", cats, "sites_sj")
    # # #note that catID is in the BB cats_merge, but only a LONG, need a DOUBLE to account for NHDPlusIDs
    # arcpy.AddField_management("sites_sj", "cat_ID", "DOUBLE")
    # arcpy.CalculateField_management("sites_sj", "cat_ID", value)
    # sites_fc = os.path.join(arcpy.env.workspace, "sites_sj")
    # count = arcpy.GetCount_management(sites_fc)
    # print('{} has {} records'.format(sites_fc, count[0]))
    # sites_lst.append(sites_fc)
    cats_lst.append(cats)

print(sites_lst)
cats_outfile = "W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb\\all_cats"
arcpy.Merge_management(cats_lst, cats_outfile)

In [103]:
## DM - Picked up here and modified to match local data
### Also want to intersect sites with maximum flow accumulation in small (30 meters) buffer.


import arcpy
import pandas as pd
import os
# Setting outgdb to location created in Covariate notebook but any temp gdb will work
outgdb = r"D:\\GIS_temp\\AKSSF_land_met\\AKSSF_land_met.gdb"
# Set data dir = to folder with regional subfolders and gdbs
data_dir = r"D:\GIS_Temp\AKSSF"
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338) #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
regions  = arcpy.ListWorkspaces(workspace_type="Folder")

# Create lists to store paths to data copies to be merged
site_lst = []
cats_lst = []

# Walk through gdbs and select sites_sj and merge into single fc
# Seperate data by
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']
# Loop through all processing areas
rois = nhdplus_dat + tauDem_dat
for roi in rois:
    # Loop through regional folders
    for region in regions:
        if roi in str(region):
            if roi in nhdplus_dat:
                cat_cur_fields = ['cat_ID_txt', 'NHDPlusID',"cat_ID_con"]
                sj_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
                wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
                print (f'{roi} in {nhdplus_dat} AKSSF list, using cat_fields {cat_cur_fields} and watershed fields {sj_cur_fields}')
                print('----------')
            # Set data and variables unique to regions with TauDEM Data
            elif roi in tauDem_dat:
                # Fields for update cursor
                cat_cur_fields = ['cat_ID_txt', 'gridcode',"cat_ID_con"]
                sj_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
                wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
                print (f'{roi} in {tauDem_dat} TauDEM list, using cat_fields {cat_cur_fields} and watershed fields {sj_cur_fields}')
                print('----------')
            print(f'{roi} using data from {region} folder')
            arcpy.env.workspace = region
            gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
            walk = arcpy.da.Walk(region, datatype = 'FeatureClass')
            for dirpath, dirnames, filenames in walk:
                for filename in filenames:
                    # Create list of watersheds
                    if filename == 'sites_sj':
                        print(f'{dirpath},{filename}')
                        sitesource = os.path.join(dirpath, filename)
                        sitename = roi +'_' +filename
                        sitepath = os.path.join(outgdb,sitename)
                        # Make local copy projected in AKAlbers
                        if not arcpy.Exists(sitepath):
                            print(f'Copying {sitename}...')
                            print('----------')
                            sitecopy = arcpy.FeatureClassToFeatureClass_conversion(sitesource,outgdb,sitename)
                        else:
                            print(f'{sitepath} already created')
                            sitecopy = sitepath
                            print('----------')
                        site_lst.append(sitecopy)
                        sitefieldnames = []
                        sitelstFields = arcpy.ListFields(sitecopy)
                        for field in sitelstFields:
                            sitefieldnames.append(field.name)
                        if str(sj_cur_fields[0]) in sitefieldnames:
                            print (f'{sj_cur_fields[0]} field already in dataset')
                            print('----------')
                        else:
                            print (f'Adding {sj_cur_fields[0]} field to watershed dataset {sitecopy}')
                            print('----------')
                            # add cat_ID_txt field and concat cat_ID + region
                            arcpy.AddField_management(sitecopy, str(sj_cur_fields[0]),field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(sitecopy, sj_cur_fields[0:2]) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[0] = strval.replace('.0','')
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)
                        if str(sj_cur_fields[2]) in sitefieldnames:
                            print (f'{sj_cur_fields[2]} field already in dataset {sitecopy}')
                            print('----------')
                        else:
                            print (f'Adding {sj_cur_fields[2]} field to watershed dataset {sitecopy}')
                            print('----------')
                            # add cat_ID_con field and concat cat_ID + region
                            arcpy.AddField_management(sitecopy, str(sj_cur_fields[2]),field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(sitecopy, sj_cur_fields) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[2] = str(roi) +'_'+ strval.replace('.0','')
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)
                                        # Set merged watersheds dataset
                    elif 'wtds_merge' == filename:
                        wtdsource = os.path.join(dirpath, filename)
                        wtdname = roi +'_'+filename
                        wtdpath = os.path.join(outgdb,wtdname)

                        if not arcpy.Exists(wtdpath):
                            print(f'Copying {wtdname}...')
                            print('----------')
                           # Make local copy projected in AKAlbers
                            wtd_merge = arcpy.FeatureClassToFeatureClass_conversion(wtdsource, outgdb,wtdname)
                        else:
                            print(f'Merged watershed dataset {wtdpath} already created...')
                            print('----------')
                            wtd_merge = wtdpath
                        wtdfieldnames = []
                        wtdlstFields = arcpy.ListFields(wtd_merge)
                        for field in wtdlstFields:
                            wtdfieldnames.append(field.name)
                        if str(wtd_cur_fields[0]) in wtdfieldnames:
                            print (f'{wtd_cur_fields[0]} field already in dataset')
                            print('----------')
                        else:
                            print (f'Adding {wtd_cur_fields[0]} field to watershed dataset {wtd_merge}')
                            print('----------')
                            # add cat_ID_txt field and concat cat_ID + region
                            arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[0]),field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields[0:2]) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[2] = strval.replace('.0','')
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)
                        if str(wtd_cur_fields[2]) in wtdfieldnames:
                            print (f'{wtd_cur_fields[2]} field already in dataset {wtd_merge}')
                            print('----------')
                        else:
                            print (f'Adding {wtd_cur_fields[2]} field to watershed dataset {wtd_merge}')
                            print('----------')
                            # add cat_ID_con field and concat cat_ID + region
                            arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[2]),field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[2] = str(roi) +'_'+ strval.replace(".0","")
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)


                    # Select catch_int fc (catchments of interest for region) and make a copy
                    elif 'cats_intersect' == filename:
                        # Make local copy projected in AKAlbers
                        catssource = os.path.join(dirpath,filename)
                        catsname = roi+"_"+filename
                        catspath = os.path.join(outgdb,catsname)
                        if not arcpy.Exists(catspath):
                            print(f'Copying {catsname}...')
                            print('----------')
                            cats = arcpy.FeatureClassToFeatureClass_conversion(catssource, outgdb,catsname)
                        else:
                            print (f'{catspath} already created')
                            print('----------')
                            cats = catspath
                        cats_lst.append(cats)
                        catlstfields = arcpy.ListFields(cats)
                        catfieldnames = []
                        for field in catlstfields:
                            catfieldnames.append(field.name)
                        if str(cat_cur_fields[0]) in catfieldnames:
                            print (f'{cat_cur_fields[0]} field already in dataset {cats}')
                            print('----------')
                        else:
                            print (f'Adding {cat_cur_fields[0]} field to catchment dataset {cats}')
                            print('----------')
                            # add cat_ID_txt field
                            arcpy.AddField_management(cats, str(cat_cur_fields[0]), field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(cats, cat_cur_fields[0:2]) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[0] = strval.replace('.0','')
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)
                        if str(cat_cur_fields[2]) in catfieldnames:
                            print (f'{cat_cur_fields[2]} field already in dataset {cats}')
                            print('----------')
                        else:
                            print (f'Adding {cat_cur_fields[2]} field to catchment dataset {cats}')
                            print('----------')
                            # add cat_ID_txt field & cat_ID + region concat field
                            arcpy.AddField_management(cats,str(cat_cur_fields[2]),field_type='TEXT')
                            # populate cat_ID_con
                            with arcpy.da.UpdateCursor(cats, cat_cur_fields) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[2] = str(roi) +'_'+ strval.replace(".0","")
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)
print(cats_lst)
print(site_lst)
outcats = os.path.join(outgdb, "all_cats_int_merge")
outsites = os.path.join(outgdb,"all_sites_sj_merge")
sitesmerge = arcpy.Merge_management(site_lst, outsites, "", "ADD_SOURCE_INFO")
catsmerge = arcpy.Merge_management(cats_lst, outcats, "", "ADD_SOURCE_INFO")
print(f"Merge complete: {sitesmerge}"
      f"{catsmerge} created...")

Cook_Inlet in ['Cook_Inlet', 'Copper_River'] AKSSF list, using cat_fields ['cat_ID_txt', 'NHDPlusID', 'cat_ID_con'] and watershed fields ['cat_ID_txt', 'cat_ID', 'cat_ID_con']
----------
Cook_Inlet using data from D:\GIS_temp\AKSSF\Cook_Inlet folder
D:\\GIS_temp\\AKSSF_land_met\\AKSSF_land_met.gdb\Cook_Inlet_cats_intersect already created
----------
cat_ID_txt field already in dataset D:\\GIS_temp\\AKSSF_land_met\\AKSSF_land_met.gdb\Cook_Inlet_cats_intersect
----------
cat_ID_con field already in dataset D:\\GIS_temp\\AKSSF_land_met\\AKSSF_land_met.gdb\Cook_Inlet_cats_intersect
----------
D:\GIS_temp\AKSSF\Cook_Inlet\Cook_Inlet.gdb,sites_sj
D:\\GIS_temp\\AKSSF_land_met\\AKSSF_land_met.gdb\Cook_Inlet_sites_sj already created
----------
cat_ID_txt field already in dataset
----------
cat_ID_con field already in dataset D:\\GIS_temp\\AKSSF_land_met\\AKSSF_land_met.gdb\Cook_Inlet_sites_sj
----------
Merged watershed dataset D:\\GIS_temp\\AKSSF_land_met\\AKSSF_land_met.gdb\Cook_Inlet_wtds_me

In [110]:
import arcpy
import os
from arcpy.sa import *
# Setting outgdb to location created in Covariate notebook but any temp gdb will work
outgdb = r"D:\\GIS_temp\\AKSSF_land_met\\AKSSF_land_met.gdb"
# Set data dir = to folder with regional subfolders and gdbs
data_dir = r"D:\GIS_Temp\AKSSF"
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338) #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
regions  = arcpy.ListWorkspaces(workspace_type="Folder")

#buffer sites
bufpath = os.path.join(outgdb,"sites_buffer")
buffer = arcpy.Buffer_analysis(sitesmerge, bufpath, "30 meters")

# this makes sure the buffer does not extend outside of the catchment - cannot use memory/ object as input for zonal stats
# must write to gdb 
clip_path = os.path.join(outgdb,'all_buffsites_clip')
clip_outfile = arcpy.Clip_analysis(buffer, catsmerge, clip_path)


#zonal stats on buffer intersection with flow accumulation grids (need to loop through regions for this)
# to get maximum flow accumulation bc some points may not exactly fall on stream grid
tbl_lst = []
# Seperate data by
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Loop through all processing areas
rois = nhdplus_dat + tauDem_dat

for roi in rois:
    # Loop through regional folders
    for region in regions:
        arcpy.env.workspace = region
        raslist = arcpy.ListRasters()
        if roi in str(region):
            if roi in nhdplus_dat:
                cat_cur_fields = ['cat_ID_txt', 'NHDPlusID',"cat_ID_con"]
                cf = 25
                print (f'{roi} in {nhdplus_dat} AKSSF list, using cat_fields {cat_cur_fields} and spatial join fields {sj_cur_fields}')
                print('----------')
            # Set data and variables unique to regions with TauDEM Data
            elif roi in tauDem_dat:
                # Fields for update cursor
                cat_cur_fields = ['cat_ID_txt', 'gridcode',"cat_ID_con"]
                cf = 100
                print (f'{roi} in {tauDem_dat} TauDEM list, using cat_fields {cat_cur_fields} and spatial join fields {sj_cur_fields}')
                print('----------')
            print(f'{roi} using data from {region} folder')
            print('--------')
            fac = os.path.join(region, 'fac.tif')
            if arcpy.Exists(fac):
                print (f'Flow accumulation grid {fac} selected for {roi}')
            else:
                print (f'No flow accumulation raster for {region}')


            outtable = os.path.join(outgdb,(roi + "_maxfac"))
            print(f'Calculating {roi} catchment Max flow accumulation stats...')
            arcpy.env.snapRaster = fac
            arcpy.env.cellSize = fac
            try:
                field = 'cat_ID_con'
                operator = 'LIKE'
                value = roi
                sba = """"{}" {} '{}%'""".format(field, operator, value)
                clip_select = arcpy.SelectLayerByAttribute_management(clip_outfile,'NEW_SELECTION',sba)
                zon_start = time.time()
                cat_facMax_table = ZonalStatisticsAsTable(in_zone_data = clip_select ,
                                                            zone_field = cat_cur_fields[2],
                                                            in_value_raster = fac,
                                                            out_table = outtable,
                                                            statistics_type='MAXIMUM'
                                                            )
                # Add region identifier field for catchment table
                arcpy.AddField_management(cat_facMax_table,'region',field_type='TEXT')
                # Add cat_ID_Con field
                arcpy.AddField_management(cat_facMax_table,'cat_ID_txt',field_type='TEXT')
                # Update fields
                # Add upstream area as calculated from Max flow acc zonal stats for catchment table
                arcpy.AddField_management(cat_facMax_table,'site_acc_sqKm',field_type='DOUBLE')

                with arcpy.da.UpdateCursor(cat_facMax_table,['region','cat_ID_txt','cat_ID_con', 'MAX', 'site_acc_sqKm']) as cur:
                    for row in cur:
                        strval = str(row[1])
                        row[0] = roi
                        row[1] = strval.replace('.0','')
                        row[4] = (cf * row[3])/1000000
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # rename MAX field
                arcpy.AlterField_management(cat_facMax_table,"MAX",'site_max_acc','site_max_acc')
                zon_stop = time.time()
                zon_time = int (zon_stop - zon_start)
                print(f'Max Flow zonal stats for {roi} Elapsed time: ({datetime.timedelta(seconds=zon_time)})')
                print('----------')

                tbl_lst.append(outtable)
            except:
                e = sys.exc_info()[1]
                print(e.args[0])
                arcpy.AddError(e.args[0])

Cook_Inlet in ['Cook_Inlet', 'Copper_River'] AKSSF list, using cat_fields ['cat_ID_txt', 'NHDPlusID', 'cat_ID_con'] and spatial join fields ['cat_ID_txt', 'cat_ID', 'cat_ID_con']
----------
Cook_Inlet using data from D:\GIS_temp\AKSSF\Cook_Inlet folder
--------
Flow accumulation grid D:\GIS_temp\AKSSF\Cook_Inlet\fac.tif selected for Cook_Inlet
Calculating Cook_Inlet catchment Max flow accumulation stats...
Max Flow zonal stats for Cook_Inlet Elapsed time: (0:22:41)
----------
Copper_River in ['Cook_Inlet', 'Copper_River'] AKSSF list, using cat_fields ['cat_ID_txt', 'NHDPlusID', 'cat_ID_con'] and spatial join fields ['cat_ID_txt', 'cat_ID', 'cat_ID_con']
----------
Copper_River using data from D:\GIS_temp\AKSSF\Copper_River folder
--------
Flow accumulation grid D:\GIS_temp\AKSSF\Copper_River\fac.tif selected for Copper_River
Calculating Copper_River catchment Max flow accumulation stats...
Max Flow zonal stats for Copper_River Elapsed time: (0:00:53)
----------
Bristol_Bay in ['Bristol

In [111]:
tbl_lst

['D:\\\\GIS_temp\\\\AKSSF_land_met\\\\AKSSF_land_met.gdb\\Cook_Inlet_maxfac',
 'D:\\\\GIS_temp\\\\AKSSF_land_met\\\\AKSSF_land_met.gdb\\Copper_River_maxfac',
 'D:\\\\GIS_temp\\\\AKSSF_land_met\\\\AKSSF_land_met.gdb\\Bristol_Bay_maxfac',
 'D:\\\\GIS_temp\\\\AKSSF_land_met\\\\AKSSF_land_met.gdb\\Kodiak_maxfac',
 'D:\\\\GIS_temp\\\\AKSSF_land_met\\\\AKSSF_land_met.gdb\\Prince_William_Sound_maxfac']

In [113]:
# Merge flow accumulation outtables
out_table_merge = os.path.join(outgdb,"AKSSF_site_sj_maxfac")
flowmerge = arcpy.Merge_management(tbl_lst,out_table_merge,"","ADD_SOURCE_INFO")
print("Merge all maxfac tables completed")
# Merge watersheds and recalculate area_km2 to join back to sites sj - Not all sites had these fields joined
wtds_merge_list = []
arcpy.env.workspace = outgdb
for fc in arcpy.ListFeatureClasses('*wtds_merge'):
    print(fc)
    arcpy.CalculateField_management(fc,"Area_km2",expression="!SHAPE.area@SQUAREKILOMETERS!")
    fieldlist = arcpy.ListFields(fc)
    wtds_merge_list.append(fc)

wtds_merge_path = os.path.join(outgdb,"all_akssf_wtds")
wtds_merge = arcpy.Merge_management(wtds_merge_list,wtds_merge_path,"","ADD_SOURCE_INFO")

Merge all maxfac tables completed
Cook_Inlet_wtds_merge
Copper_River_wtds_merge
Bristol_Bay_wtds_merge
Kodiak_wtds_merge
Prince_William_Sound_wtds_merge


In [114]:
#Join fields from flow acc zonal stats and wtds merge to compare watershed area
flow_join_fields = ['site_max_acc','site_acc_sqKm']
wtds_join_fields = ['Area_km2']
arcpy.JoinField_management(sitesmerge,sj_cur_fields[2], out_table_merge,sj_cur_fields[2],flow_join_fields)
arcpy.JoinField_management(sitesmerge,sj_cur_fields[2], wtds_merge,sj_cur_fields[2],wtds_join_fields)
arcpy.AddField_management(sitesmerge,"area_diff",field_type="DOUBLE")
with arcpy.da.UpdateCursor(sitesmerge,["area_diff",'Area_km2','site_acc_sqKm']) as cur:
    for row in cur:
        row[0] = row[1] - row[2]
        cur.updateRow(row)
    del(row)
del(cur)

In [120]:
# Make site merge df
sitemerge_df = pd.DataFrame()
sitemerge_field_list = []
keepfields = ['SiteID','cat_ID_con','site_max_acc','site_acc_sqKm','Area_km2','area_diff' ]
for field in arcpy.ListFields(sitesmerge):
    sitemerge_field_list.append(field.name)
    print(field.name)
sitesmerge_arr = arcpy.da.TableToNumPyArray(sitesmerge,keepfields)
sitemerge_df = pd.DataFrame(sitesmerge_arr)
sitemerge_df = sitemerge_df.set_index('cat_ID_con')
dfs.append(sitemerge_df)
sitemerge_df

OBJECTID
Shape
Join_Count
TARGET_FID
SiteID
Wtrbdy_n
Agnc_ID
SourcNm
Cntct_p
Cntct_m
Cntct_t
Snsr_cc
file_nm
seq_id
Snsr_Pl
Wtrbdy_t
Verified
Comments
DM_Notes
Sn_QAQC
HUC8
Name
NHDPlusID
SourceFC
GridCode
AreaSqKm
VPUID
Catch_ID
cat_ID
cat_ID_txt
cat_ID_con
HUC8_ID
catID
proc_reg
MERGE_SRC
site_max_acc
site_acc_sqKm
Area_km2
area_diff


,SiteID,site_max_acc,site_acc_sqKm,Area_km2,area_diff
cat_ID_con,,,,,
Cook_Inlet_75005400001203,CIK_48,11297657,282.44,282.47,0.02
Cook_Inlet_75005400013272,CIK_47,427430,10.69,10.78,0.10
Cook_Inlet_75004300006239,CIK_18,2224162,55.60,56.02,0.42
Cook_Inlet_75004300007331,CIK_19,2760921,69.02,69.45,0.43
Cook_Inlet_75004300002289,CIK3,309939,7.75,8.15,0.41
...,...,...,...,...,...
Prince_William_Sound_36645,USFS_ERB Creek,35947,3.59,4.13,0.53
Prince_William_Sound_37815,USFS_Solf Lake Outlet Creek,52568,5.26,5.80,0.54
Prince_William_Sound_42563,15219000,122628,12.26,12.80,0.53
